In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [106]:
# Make a request to the website
url = 'https://user.wartale.com/guide.asp?page=1&action=2&id=492&spec=0&mix=0&age=0&box=2'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
print(response)

<Response [200]>


In [107]:
import json

# Existing code to extract values
soup = BeautifulSoup(response.content, 'html.parser')

def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

item_box_header = soup.find('div', class_='item_box_header')
item_name = item_box_header.contents[0].strip()
item_id = soup.find("div", class_="item_box_img").img["alt"]

tr_indices = [1, 4, 7]
data = {
    "item_name": item_name,
    "item_id": item_id
}

for index in tr_indices:
    tr = soup.find_all('tr')[index]
    data.update(extract_values(tr))

# Write data to a JSON file
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

print("Data stored in data.json file.")

Data stored in data.json file.


In [4]:
url2 = 'https://user.wartale.com/index.asp?page=30'
response2 = requests.get(url2)
soup = BeautifulSoup(response2.content, 'html.parser')

In [12]:


first_span = soup.find('span')
first_span


<span class="links_box">
<a href="#" id="type_1" onclick="showSubtypes(1);">Weapons</a> | 
						<a href="#" id="type_2" onclick="showSubtypes(2);">Defenses</a> | 
						<a href="#" id="type_3" onclick="showSubtypes(3);">Accessories</a>
</span>

In [103]:

data = []
span = soup.find('span')
for a_tag in span.find_all('a'):
    data.append({
        'id_type': a_tag.get('id'),
        'category': a_tag.text
    })

df = pd.DataFrame(data)

df['subtypes'] = df['id_type'].apply(lambda x: f'sub{x}')

df

,id_type,category,subtypes
0,type_1,Weapons,subtype_1
1,type_2,Defenses,subtype_2
2,type_3,Accessories,subtype_3


In [104]:
subtypes = df['subtypes'].tolist()
subtypes

['subtype_1', 'subtype_2', 'subtype_3']

In [105]:
data2 = []
for stype in subtypes:
    a = soup.find('span', id = stype)
    for a_tag in a.find_all('a'):
        data2.append({
            'subtypes': stype,
            'id_subtype': a_tag.get('id'),
            'type': a_tag.text
        })
df2 = pd.DataFrame(data2)    
df2    

,subtypes,id_subtype,type
0,subtype_1,sub_2,Axes
1,subtype_1,sub_5,Bows
2,subtype_1,sub_4,Claws
3,subtype_1,sub_19,Daggers
4,subtype_1,sub_21,Fists
5,subtype_1,sub_7,Hammers
6,subtype_1,sub_6,Javelins
7,subtype_1,sub_20,Phantoms
8,subtype_1,sub_8,Scythes
9,subtype_1,sub_3,Swords


In [69]:
xd = soup.find('span', id = 'subtype_1')
xd

<span class="links_box" id="subtype_1" style="display: none;">
<a href="#" id="sub_2" onclick="showItems(2);">Axes</a> | 
						<a href="#" id="sub_5" onclick="showItems(5);">Bows</a> | 
						<a href="#" id="sub_4" onclick="showItems(4);">Claws</a> | 
						<a href="#" id="sub_19" onclick="showItems(19);">Daggers</a> | 
						<a href="#" id="sub_21" onclick="showItems(21);">Fists</a> | 
						<a href="#" id="sub_7" onclick="showItems(7);">Hammers</a> | 
						<a href="#" id="sub_6" onclick="showItems(6);">Javelins</a> | 
						<a href="#" id="sub_20" onclick="showItems(20);">Phantoms</a> | 
						<a href="#" id="sub_8" onclick="showItems(8);">Scythes</a> | 
						<a href="#" id="sub_3" onclick="showItems(3);">Swords</a> | 
						<a href="#" id="sub_1" onclick="showItems(1);">Wands &amp; Staffs</a>
</span>

In [87]:
data2 = []



aaaa = 'subtype_1'
xd = soup.find('span', id = aaaa )




for a_tag in xd.find_all('a'):
    data2.append({
        'subtype': aaaa,
        'id': a_tag.get('id'),
        'onClick': a_tag.get('onclick'),
        'type': a_tag.text
    })

df2 = pd.DataFrame(data2)    
df2

,subtype,id,onClick,type
0,subtype_1,sub_2,showItems(2);,Axes
1,subtype_1,sub_5,showItems(5);,Bows
2,subtype_1,sub_4,showItems(4);,Claws
3,subtype_1,sub_19,showItems(19);,Daggers
4,subtype_1,sub_21,showItems(21);,Fists
5,subtype_1,sub_7,showItems(7);,Hammers
6,subtype_1,sub_6,showItems(6);,Javelins
7,subtype_1,sub_20,showItems(20);,Phantoms
8,subtype_1,sub_8,showItems(8);,Scythes
9,subtype_1,sub_3,showItems(3);,Swords
